# covid-19 crisis

### Introduction

As the world grapples with the challenges posed by the COVID-19 pandemic, understanding the intricacies of the virus's spread and its effects on various demographics becomes crucial in formulating effective response strategies. South Korea, given its proximity to the initial outbreak and its robust healthcare system, offers valuable insights into the pandemic's progression and containment.

The provided dataset encompasses detailed patient-level data from South Korea, shedding light on various aspects of the pandemic — from the source of infection to the time it took for symptom onset and subsequent confirmation. Covering a range of provinces and cities, the data encapsulates diverse demographics, allowing for a comprehensive analysis that can aid in formulating targeted response strategies.

With parameters such as age, gender, infection source, and days between symptom onset and confirmation, this analysis seeks to unearth patterns, identify at-risk demographics, and gauge the effectiveness of existing measures. By leveraging this data, our aim is to fortify our homeland's preparedness for potential subsequent waves and refine our approach in the ongoing "Corona Fight."

Understanding the depth and implications of this data is imperative. As we navigate through its layers, we'll extract key insights, paving the way for evidence-backed decisions that prioritize the well-being of our citizens and the resilience of our healthcare systems.

**Source of data:** [KCDC (Korea Centers for Disease Control & Prevention](https://www.kaggle.com/datasets/kimjihoo/coronavirusdataset)

### Data Overview
The dataset in focus is an exhaustive compilation of patient-level data related to the COVID-19 cases in South Korea. Derived from a reputable source, the data encompasses a duration from January to June 2020, a critical period in the pandemic's timeline.

* Personal Information: Includes patient ID, gender, age, country of residence, and specific location data (province and city).
* Infection Details: Captures the source of the infection, whether it was an overseas inflow, contact with an existing patient, or other categories. Also includes specific details on which patient an individual was infected by (if applicable) and the number of contacts they had.
* Medical Timeline: Details symptom onset date, the date of confirmation, the date of release (recovery), and the date of decease (if applicable).
* State: Captures the current health state of the patient, such as isolated, released, or deceased.
* Derived Metrics: These metrics include days between symptom onset and confirmation, days between confirmation and release, and days between confirmation and decease.

Preprocessing Steps Undertaken: Handling missing values, date conversion, computation of derived metrics and categorization.

In [1]:
# Importing libraries

from my_func import *
from collections import Counter
import re

pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
# Print out the available datasets

In [ ]:
ls datasets/covid

In [ ]:
# Import Patient Level dataset. 

df = pd.read_csv('datasets/covid/PatientInfo.csv')
df.head()

In [ ]:
df.shape[0], df.shape[1]

In [ ]:
# Printing out different columns
df.columns

In [ ]:
# Changing columns names
df.columns = ['ID', 'Sex', 'Age', 'Country', 'Province', 'City', 'Infection Case', 'Infected By', 'Contact Number',
             'Symptom Onset Date', 'Confirmed Date', 'Released Date', 'Deceased Date', 'State']

In [ ]:
df.columns

**Type of features**

Categorical Features = Patient ID, Sex, Age groups, Country, Province, City, Infection Case, Infected_By, State

Continuous Features = Contact Number


### Cleaning Data

In [ ]:
df.dtypes

In [ ]:
# Define data types

df[['Sex', 'Country', 'Province', 'City', 'Infection Case', 'State']] = df[['Sex', 'Country', 'Province', 'City', 'Infection Case', 'State']].astype("string")

In [ ]:
# Check missing values

df.isnull().sum()

In [ ]:
# Visualize missingness with modified missingno library matrix function

fig, ax = plt.subplots(figsize=(10, 5))
missing_matrix(df, ax=ax, sparkline=False)
plt.show()

In [ ]:
# EXPLICACION MISSING VALUES

**Handling missing values**

In [ ]:
# We will first handle missing cities finding the most common 
# city within the same province, country and state
# Applying the function to fill missing city values from my_func.py file

df['City'] = df.apply(
    lambda row: fill_missing_value(row, df, 'City', 
                                   ['Province', 'Country', 'State']), axis=1).astype('string')

In [ ]:
# We want to now observe if there is a any correlation between sex and the province feature 
# then we will understand if we can fill in empty values based on known values

categorical_columns = ['Province', 'Country', 'City']
chi2_correlation('Sex', categorical_columns, df)

* Province-Sex Association: The p-value (1.4154599552335896e-05) is very small, indicating that there is a statistically significant association between the province and sex of the individuals. This tell us that the distribution of sexes is not the same within all provinces.

* Country-Sex Association: The p-value (0.059418440980756944) is larger than 0.05, we do not have enough evidence to say there is a statistically significant association between the country and the sex of the individuals.

* City-Sex Association: The p-value (2.5807447933143334e-06) is extremely small, indicating a statistically significant association between the city and sex of the individuals. This suggests the distribution of sexes is not the same across all cities.

In [ ]:
# we want to now fill in empty sex values based on the city mode

# group the dataframe by 'City' and calculate the mode of the 'Sex' column
mode_by_city = df.groupby('City')['Sex'].apply(lambda x: x.mode()[0] if not x.mode().empty else None)

# fill in missing values in 'Sex' based on the mode of the corresponding 'City'
df['Sex'] = df.apply(lambda row: mode_by_city[row['City']] if pd.isnull(row['Sex']) else row['Sex'], axis=1).astype('string')

# replace any remaining missing values with the overall mode
mode_sex = df['Sex'].mode()[0]
df['Sex'].fillna(mode_sex, inplace=True)

In [ ]:
pie_chart(df, 'Sex')

In [ ]:
# we want to now fill in empty age vales based on the city mode.

# group the dataframe by 'province' and calculate the mode of the 'age' column
mode_by_city = df.groupby('City')['Age'].apply(lambda x: x.mode()[0] if not x.mode().empty else None)

# fill in missing values in 'Sex' based on the mode of the corresponding 'City'
df['Age'] = df.apply(lambda row: mode_by_city[row['City']] if pd.isnull(row['Age']) else row['Age'], axis=1).astype('string')

# replace any remaining missing values with the overall mode
mode_sex = df['Age'].mode()[0]
df['Age'].fillna(mode_sex, inplace=True)

In [ ]:
# To handle the infecion case part, I will create a new category to fill in empty spaces for the Unknown ones.
df['Infection Case'] = df['Infection Case'].fillna('Unknown').astype('string')

In [ ]:
# we will fill in the infected by values empties with unknown and the na also
# I assume missing values might be unknown by the patient
df['Infected By'] = df['Infected By'].apply(lambda x: convert_to_float(x, 10))

In [ ]:
# Replace 'nan' and '-' with actual NaN values
df['Contact Number'] = df['Contact Number'].apply(lambda x: float(np.nan) if x == '-' else x)

In [ ]:
df['Contact Number'] = df['Contact Number'].apply(lambda x: convert_to_float(x, 5))

Sympton Onset Date data has a large number of missing values that could tell us that there are many asymptomatic
cases or cases where symptom onset wasn't tracked. 
Since most of the column shows as missing value I will assume that people got tested soon after they start showing symptoms I'm gonna use the confirmed date to fill in the empty values. 

In [ ]:
# Fill missing 'Symptom Onset Date' with 'Confirmed Date' for applicable rows
df.loc[df['Symptom Onset Date'].isnull(), 'Symptom Onset Date'] = df.loc[df['Symptom Onset Date'].isnull(), 'Confirmed Date']
df['Symptom Onset Date'] = pd.to_datetime(df['Symptom Onset Date'], errors='coerce', format='%Y-%m-%d')

In [ ]:
# We will drop rows with confirmed_date missing data since is just a very little amount
# Drop rows where 'confirmed_date' is missing and convert column to datetime dataframe

df.dropna(subset=['Confirmed Date'], inplace=True)
df['Confirmed Date'] = pd.to_datetime(df['Confirmed Date'], format='%Y-%m-%d')
histogram(df, 'Confirmed Date', bins=50)

In [ ]:
df['Released Date'] = pd.to_datetime(df['Released Date'], format='%Y-%m-%d')
df['Deceased Date'] = pd.to_datetime(df['Deceased Date'], format='%Y-%m-%d')

### Feature Engineering

Adding new time-related columns.

- Days between symptom onset and confirmation for those that we had data, since the ones that we didn't know we assumed that the patient had no symptons or knowledge of having COVID-19 so the symptons date would be same as the confirmation date. That day we will have insights on which patients had symtoms and when the patient got confirmed at the hospital.

- Days between confirmation and release / decease dates to later on plan a little bit resources with healthcare resource planning.

In [ ]:
# Calculation days between symptons onset and confirmation date
df['Days between Symptom Onset and Confirmation'] = (df['Confirmed Date'] - df['Symptom Onset Date']).dt.days

In [ ]:
# calculation days between confirmation date and released date 
df['Days between Confirmation and Release'] = (df['Released Date'] - df['Confirmed Date']).dt.days

In [ ]:
# calculation days between confirmation date and deceased date 
df['Days between Confirmation and Deceased'] = (df['Deceased Date'] - df['Confirmed Date']).dt.days

In [ ]:
# Concatenate all the strings in the 'Infection Case' column into a single text
all_text = ' '.join(df['Infection Case'].astype(str))

# Tokenize the text into individual words
words = re.findall(r'\b\w+\b', all_text.lower())

# Count the frequency of each word
word_freq = Counter(words)

# Sort the words based on their frequency in descending order
sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)

# Display the most repeated words and their frequencies
# print("Most repeated words:")
# for word, freq in sorted_words:
#    print(f"{word}: {freq}")

In [ ]:
df['Infection Case'] = df['Infection Case'].str.lower()


# Create a new column for the category
df['Infection Case Category'] = df['Infection Case'].apply(categorize_case)

### Summary Statistics

Calculating the mean, median, mode, minimum, maximum, and standard deviation for numerical columns, and counts or frequency distributions for categorical columns.

#### Demomgraphic Analysis

In [ ]:
# Calculating frequency distributions for categorical columns starting with demographic columns
# Age groups, genders, status of the infection

df['Age'] = df['Age'].str.rstrip('s').astype(int)

In [ ]:
counterplot('Age', df, xlabel_string="Age Groups")

In [ ]:
counterplot_hue(df=df, target_column = "Age", hue= "Sex", xlabel = "Age Groups", title="Age per sex distribution")

In [ ]:
df['Age'].describe()

So far we can see the most vulnerable groups are people in their 20s and 50s. Now let's differenciate which are males and which are females.

In [ ]:
pie_chart(df, 'State')

In [ ]:
num_deceased = len(df[df['State'] == 'deceased'])
total_confirmed_cases = len(df)
fatality_rate = (num_deceased / total_confirmed_cases) * 100
fatality_rate

In [ ]:
num_recovered = len(df[df['State'] == 'released'])
recovery_rate = (num_recovered / total_confirmed_cases) * 100
recovery_rate

The recovery rate is promising since more than half of the confirmed cases have recovered.
The fatality rate, while relatively low, still signifies the importance of preventive measures.

Let's go further by analysis the relationship between ages, sex and status of the disease. 

In [ ]:
counterplot_hue(df=df, target_column = "State", hue= "Age", xlabel = "Status", title="Status per Age")

#### Geographic Analysis

In [ ]:
# Create province cases dataframe by grouping province by number of cases
province_cases = df.groupby('Province').size().reset_index(name='Total Cases')
province_cases.at[11, 'Province'] = 'Jeju'

In [ ]:
# Import population by regions in South Korea
pop_df = pd.read_csv('datasets/population_density_korea_south_regions.csv')
pop_df.rename(columns={'By administrative divisions' : "Province"}, inplace=True)

In [ ]:
# Rename a province name so their are matching
pop_df.at[8, 'Province'] = 'Sejong'
pop_df.at[17, 'Province'] = 'Jeju'
pop_df = pop_df.iloc[1:]
pop_df = pop_df.reset_index(drop=True)

In [ ]:
# Merge the province population and province cases datasets by province name
province_cases_population = province_cases.merge(pop_df, on='Province')

# Reset the index to an unified numeration and print it out
province_cases_population.reset_index(drop=True, inplace=True)

In [ ]:
create_map('Province', 'shapefiles/dk009rq9138.shp', '2019', 'Population Density in 2019', province_cases_population)

Checking further the relationship between population density, and the number of cases in different regions.

In [ ]:
create_map('Province', 'shapefiles/dk009rq9138.shp', 'Total Cases', 'Total Cases per Province in 2020', province_cases_population)

In [ ]:
create_map('Province', 'shapefiles/dk009rq9138.shp', '2019', 'Population Density in 2020', province_cases_population)

In [ ]:
# Calculate the correlation coefficient
correlation = province_cases_population['Total Cases'].corr(province_cases_population['2020'])

# Print the correlation coefficient
print("Correlation Coefficient:", correlation)

# Plot a scatter plot to visualize the relationship
plt.scatter(province_cases_population['2020'], province_cases_population['Total Cases'])
plt.xlabel('2020')
plt.ylabel('Total Cases')
plt.title('Correlation between Total Cases and Population Density')
plt.show()

Based on the correlation coefficient of 0.5089, there is a moderate positive correlation between the total cases and population density in the given dataset. This means that as the population density in 2019 & 2020, the total number of cases tends to increase as well, and vice versa. 

Calculating the incidence rate for each region by dividing the number of COVID-19 cases by the population size. This will give us a measure of the spread of the virus relative to the population size.

In [ ]:
province_cases_population['Incidence Rate'] = province_cases_population['Total Cases'] / (province_cases_population['2019'])

In [ ]:
province_cases_population

In [ ]:
create_map('Province', 'shapefiles/dk009rq9138.shp', 'Incidence Rate', 'Incidence Rate', province_cases_population)

It can be indicative of localized outbreaks or clusters of COVID-19 cases in specific regions. This can lead to a higher incidence rate despite the lower population density.

In [ ]:
province_name = 'Gyeongsangbuk-do'  # Province name to filter
outbreak = df.loc[df['Province'] == province_name]

In [ ]:
outbreak

In [ ]:
pie_chart(outbreak, 'Infection Case Category')

Among the infection cases in the outbreak province, a significant portion (57%) is categorized as 'Unknown,' indicating that the source of transmission for these cases is yet to be determined. Another substantial proportion (20.8%) falls under the category 'other', which likely encompasses various miscellaneous cases. The remaining cases (22.2%) can be attributed to specific sources, such as religious gatherings (e.g., Shincheonji Church), healthcare facilities (e.g., Gyeongsan Seorin Nursing Home and Cheongdo Daenam Hospital), and other specific locations (e.g., Gyeongsan Cham Joeun Community Center and Pilgrimage to Israel).

#### Contact and infection


In this section we will focus on the analysis of 'infection_case', 'infected_by', and 'contact_number' columns to understand how the virus was spread in the country and if we can detect any super spreader cases, sources of infection, find patterns or contact behaviors. 

In [ ]:
df['Contact Number'].describe()

The contact number column contains information on the number of contacts associated with COVID-19 cases. On average, each case had around 14 contacts, with a wide range of values from 0 to 485. The data reveals the extent of potential transmission through contact tracing efforts.

In [ ]:
histogram(df, 'Contact Number', bins=50)

Now, we will analyze the 'Contact Number' per province to identify regions with a high number of contacts. This analysis will help us understand if certain regions require more education and preventive measures to reduce contact and curb the spread of COVID-19.

In [ ]:
# Calculating the average contactt number for each province
average_contact_number_per_province = df.groupby('Province')['Contact Number'].mean().reset_index()

# Sort the data by the average 'Contact Number' in descending order
average_contact_number_per_province = average_contact_number_per_province.sort_values(by='Contact Number', ascending=False)

In [ ]:
barplot('Contact Number', 'Province', average_contact_number_per_province, 'Average contact number per province')

This analysis examines the average 'Contact Number' for each province, shedding light on potential variations in COVID-19 exposure. Daegu leads with the highest average contact number, while some provinces lack sufficient data. Understanding these patterns can help focus preventive efforts in regions with higher exposure risks as 'Daegu' and
'Jeollabuk-do' in second place but by difference. The region that needs most preventive efforts is Daegu. 

In [ ]:
df['Infection Case Category'].unique()

In [ ]:
# Exclude rows with 'Unknown' in the 'Infection Case' column
known_cases_df = df[df['Infection Case Category'] != 'Unknown']

# Count the occurrences of each infection source
infection_counts = known_cases_df['Infection Case Category'].value_counts()

In [ ]:
infection_counts

In [ ]:
counterplot('Infection Case Category', df, 'xlabel_string')

The analysis of infection cases reveals that the highest number (1610) is due to contact with patients. Additionally, there are 1092 cases classified as "Other," indicating unspecified sources. Overseas inflow accounts for 840 cases, emphasizing the importance of monitoring international travel for disease control.

In [ ]:
# Filter out rows with known sources of infection
known_sources_df = df[df['Infected By'].notna()]

# Count the occurrences of each unique source of infection
infection_counts = known_sources_df['Infected By'].value_counts()

In [ ]:
infection_counts.describe()

Infected By Analysis: Explore the 'infected_by' column to investigate the transmission patterns. Determine the number of cases that were infected by each unique source. This can help you identify super-spreader cases or individuals who contributed significantly to the transmission.

In [ ]:
infected_more_than_5 = infection_counts[infection_counts > 2]

# Get the IDs of patients who infected more than 5 people
infected_more_than_5_ids = infected_more_than_5.index.tolist()

In [ ]:
# Create a DataFrame containing only the rows with the selected IDs
selected_rows_df = df[df['ID'].isin(infected_more_than_5_ids)]

pie_chart(selected_rows_df, 'Infection Case Category')

#### Time Analysis

* From Symptom Onset to Confirmation: Testing efficiency and the possible presence of asymptomatic carriers.
    
* From Confirmation to Release/Decease: Healthcare system's response and the virus's impact.

In [ ]:
# Symptom Onset to Confirmation date

avg_days_symptons_onset_to_confirmation = df.groupby('Confirmed Date')['Days between Symptom Onset and Confirmation'].mean().reset_index()
time_series(avg_days_symptons_onset_to_confirmation, 
            'Confirmed Date', 'Days between Symptom Onset and Confirmation', 'Days between Symptom Onset and Confirmation', 'Avg number of days')

In [ ]:
avg_days_confirmation_to_release = df.groupby('Confirmed Date')['Days between Confirmation and Release'].mean().reset_index()
time_series(avg_days_confirmation_to_release, 
            'Confirmed Date', 'Days between Confirmation and Release', 'Days between Confirmation and Release', 'Avg number of days')

In [ ]:
avg_days_confirmation_to_deceased = df.groupby('Confirmed Date')['Days between Confirmation and Deceased'].mean().reset_index()
time_series(avg_days_confirmation_to_deceased, 
            'Confirmed Date', 'Days between Confirmation and Deceased', 'Days between Confirmation and Deceased', 'Avg number of days')